In [20]:
from surprise import SVD, SVDpp
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [24]:
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)
train = Dataset.load_from_file('ratings-final.csv', reader=reader)

In [25]:
algo = SVD()
#train, test = train_test_split(train, test_size=0.2)
#_, test =  train_test_split(test, test_size=0.999)

In [26]:
print("Searching for best parameters...")
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.008],
              'n_factors': [50, 100]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(train)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Searching for best parameters...
Best RMSE score attained:  0.6079803989958488
{'n_epochs': 30, 'lr_all': 0.005, 'n_factors': 50}


# algo.fit(train)
#predictions = algo.test(test)

In [5]:
accuracy.rmse(predictions, verbose=True)

RMSE: 0.6583


0.6582689804293287

In [6]:
user = []
movie = []
RealRate = []
Estimated = []
for userID, movieID, actualRating, estimatedRating, _ in predictions:
    user.append(int(userID))
    movie.append(int(movieID))
    RealRate.append(actualRating)
    Estimated.append(estimatedRating)

In [7]:
import pandas as pd
predicted = pd.DataFrame()
predicted['userId'] = user
predicted['movieId'] = movie
predicted['RealRate'] = RealRate
predicted['Estimated'] = Estimated
predicted['RealRate'] = predicted['RealRate'].astype(int)
predicted.head()

,userId,movieId,RealRate,Estimated
0,210,74,1,1.000000
1,410,68,1,1.250479
2,484,153,1,1.641460
3,60,99,1,1.531904
4,267,114,1,1.489609


In [8]:
from sklearn import preprocessing
x = predicted[['Estimated']].values.astype(float)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1,3))
predicted['Normalized'] = min_max_scaler.fit_transform(x).astype(int)
predicted['Estimated'] = predicted['Estimated'].astype(int)
predicted

,userId,movieId,RealRate,Estimated,Normalized
0,210,74,1,1,1
1,410,68,1,1,1
2,484,153,1,1,1
3,60,99,1,1,1
4,267,114,1,1,1
...,...,...,...,...,...
5428,323,173,3,1,1
5429,47,142,1,1,1
5430,297,38,1,1,1
5431,5,25,2,2,2


In [9]:
(predicted['Normalized'] - predicted['RealRate']).map(lambda x: (x == 0)).sum()

2918

In [10]:
(predicted['Estimated'] - predicted['RealRate']).map(lambda x: (x == 0)).sum()

2964

In [11]:
def check(x, y):
    if x == y:
        return 2
    elif (x, y) == (2, 3) or (x, y) == (3, 2):
        return 1
    else:
        return 0

In [13]:
sum = 0
for i in range(5433):
    sum += check(predicted.loc[predicted.index[i], 'Estimated'], predicted.loc[predicted.index[i], 'RealRate'])
sum

6274

In [14]:
sum2 = 0
for i in range(5433):
    sum2 += check(predicted.loc[predicted.index[i], 'Normalized'], predicted.loc[predicted.index[i], 'RealRate'])
sum2

6085

In [15]:
predicted['Random'] = 1
sum3 = 0
for i in range(5433):
    sum3 += check(predicted.loc[predicted.index[i], 'Random'], predicted.loc[predicted.index[i], 'RealRate'])
sum3

5544

In [16]:
predicted['RealRate'].value_counts()

1    2772
2    1767
3     894
Name: RealRate, dtype: int64

In [ ]:
12089/11124 /2